In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df= pd.read_csv('athlete_events.csv')
region_df = pd.read_csv('noc_regions.csv')

In [ ]:
df.tail()

In [ ]:
df.shape

In [ ]:
df=df[df['Season']=='Summer']

In [ ]:
df.shape

In [ ]:
df.tail()

In [ ]:
region_df.tail()

In [ ]:
df.merge(region_df,on = 'NOC', how='left')

In [ ]:
df.tail()

In [ ]:
df['region'].unique()

In [ ]:
df.isnull().sum()

In [ ]:
df.duplicated().sum()

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df.duplicated().sum()

In [ ]:
df['Medal'].value_counts()

In [ ]:
pd.get_dummies(df['Medal'])

In [ ]:
df.shape

In [ ]:
df.tail()

In [ ]:
df.groupby('NOC').sum()[['Gold','Silver','Bronze']]